In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
df_train = pd.read_csv('train.csv')


In [3]:
def add_column(I):
    list_column = I.columns
    ac = 0
    for column in list_column:
        ac = ac + I[column] 
    ac = pd.DataFrame(ac)
    return ac

In [4]:
def mult_column(I):
    I = pd.DataFrame(I)
    list_column = I.columns
    mc = 1
    for column in list_column:
        mc = mc * I[column] 
    mc = pd.DataFrame(mc)
    return mc

In [5]:
def column_cube(I):
    I = pd.DataFrame(I)
    list_column = I.columns
    c3 = 1
    for column in list_column:
        c3 = c3 * I[column]^3 
    c3 = pd.DataFrame(c3)
    return c3

In [6]:
y = df_train['Survived']
X = df_train.loc[:, df_train.columns != 'Survived']


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=45)

In [8]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer

In [10]:
age_pipeline = make_pipeline(
    (SimpleImputer(missing_values=np.nan, strategy='mean')),
    (StandardScaler()),
)
emb_pipeline = make_pipeline(
    (SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
    (OneHotEncoder(sparse=False, handle_unknown='ignore')),
)
fare_pipeline = make_pipeline(
    (SimpleImputer(missing_values=np.nan, strategy='mean')),
    (StandardScaler()),
    #(FunctionTransformer(mult_column, validate=False))
)
mult_pipeline = make_pipeline(
    (SimpleImputer(missing_values=np.nan, strategy='mean')),
    (StandardScaler()),
    (FunctionTransformer(mult_column, validate=False)),  
     )

In [11]:
transformer = make_column_transformer(
    # (transformer object, column names)
    (age_pipeline, ['Age']),
    (emb_pipeline, ['Embarked']),
    (fare_pipeline, ['Fare']),
    (OneHotEncoder(sparse=False), ['Pclass']),
    (OneHotEncoder(sparse=False), ['Sex']),
    (FunctionTransformer(add_column, validate=False), ['SibSp', 'Parch']),
    #(FunctionTransformer(mult_column, validate=False), ['Pclass', 'Fare']),
    (mult_pipeline, ['Pclass', 'Age']),
    #(mult_pipeline, ['Pclass', 'Sex']),
    #(FunctionTransformer(mult_column, validate=False), ['Pclass', 'Sex']),
    (FunctionTransformer(column_cube, validate=False), ['SibSp']),
    )

In [12]:
transformer.fit(X_train)
X_test_trans = transformer.transform(X_train)
X_test_trans = pd.DataFrame(X_test_trans)
#X_test_trans


In [13]:
X_train_trans = transformer.transform(X_train)
X_test_trans = transformer.transform(X_test)

In [14]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(n_estimators=1000, max_depth=5, n_jobs=1) # CPUs (-1 = all)

In [15]:
model_rf.fit(X_train_trans, y_train)

RandomForestClassifier(max_depth=5, n_estimators=1000, n_jobs=1)

In [16]:
y_pred_rf = model_rf.predict(X_test_trans)


In [17]:
accuracy_score(y_pred_rf, y_test)

0.8544776119402985

In [18]:
df_test = pd.read_csv('test.csv')
#X_Kaggle = df_test.drop(['Cabin'], axis=1)
X_Kaggle = df_test


In [20]:
X_test_cleaned = transformer.transform(X_Kaggle)
X_test_cleaned=pd.DataFrame(X_test_cleaned)
#X_test_cleaned.isnull().sum()

In [21]:
predictions = model_rf.predict(X_test_cleaned)
#predictions

In [22]:
submission = pd.DataFrame({'PassengerId':X_Kaggle['PassengerId'],'Survived':predictions})
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [23]:
filename = 'Titanic_Predictions_Friday2.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: Titanic_Predictions_Friday2.csv
